In [11]:
class State:
    def __init__(self, label=None, edges=None):
        self.label = label
        self.edges = edges or []

class Fragment:
    def __init__(self, start_state=None, accept_states=None):
        self.start_state = start_state
        self.accept_states = accept_states or []

In [ ]:
# | handle character after
# handle concatenation with absence of dot
# modify output format
# read json and convert to nfa
# implement nfa to minimized dfa

In [31]:
def compile_regex(regex):
    counter = -1
    stack = []
    for i in range(len(regex)):
        if regex[i] == '.':
            frag2 = stack.pop()
            frag1 = stack.pop()
            frag1.accept_states[0].edges.append((None, frag2.start_state))
            frag1.accept_states = frag2.accept_states
            stack.append(frag1)
        elif regex[i] == '|':
            frag2 = stack.pop()
            frag1 = stack.pop()
            counter=counter+1
            start_state = State()
            start_state.label=counter
            start_state.edges.append((None, frag1.start_state))
            start_state.edges.append((None, frag2.start_state))
            accept_states = frag1.accept_states + frag2.accept_states
            stack.append(Fragment(start_state, accept_states))
        elif regex[i] == '*':
            counter=counter+1
            frag = stack.pop()
            start_state = State()
            start_state.label=counter
            accept_state = State()
            counter=counter+1
            accept_state.label=counter
            start_state.edges.append((None, frag.start_state))
            start_state.edges.append((None, accept_state))
            frag.accept_states[0].edges.append((None, start_state))
            frag.accept_states[0].edges.append((None, accept_state))
            stack.append(Fragment(start_state, [accept_state]))
        
        elif regex[i] == '?':
            counter=counter+1
            frag = stack.pop()
            start_state = State()
            start_state.label=counter
            accept_state = State()
            counter=counter+1
            accept_state.label=counter

            start_state.edges.append((None, frag.start_state))
            start_state.edges.append((None, accept_state))
            frag.accept_states[0].edges.append((None, accept_state))
            stack.append(Fragment(start_state, [accept_state]))
        elif regex[i] == '+':
            counter=counter+1
            frag = stack.pop()
            start_state = State()
            start_state.label=counter
            accept_state = State()
            counter=counter+1
            accept_state.label=counter
            start_state.edges.append((None, frag.start_state))
            frag.accept_states[0].edges.append((None, start_state))
            frag.accept_states[0].edges.append((None, accept_state))
            accept_state.edges.append((None, start_state))
            stack.append(Fragment(start_state, [accept_state]))
      
        else:
            counter=counter+1
            start_state = State()
            start_state.label=counter
            accept_state = State()
            accept_state.label=counter+1
            start_state.edges.append((regex[i], accept_state))
            stack.append(Fragment(start_state, [accept_state]))
            counter=counter+1
    return stack.pop()


In [32]:
def print_nfa(nfa):
    seen = set()
    print(f"start_state: {nfa.start_state.label}")
    for accept_state in nfa.accept_states:
        print(f"accept_state: {accept_state.label}")
    print("transitions:")
    queue = [nfa.start_state]
    while queue:
        state = queue.pop(0)
        if state in seen:
            continue
        seen.add(state)
        for char, dest_state in state.edges:
            print(f"{state.label} --({char})--> {dest_state.label}")
            queue.append(dest_state)



In [33]:
regex = 'a*'
nfa = compile_regex(regex)
print_nfa(nfa)

start_state: 2
accept_state: 3
transitions:
2 --(None)--> 0
2 --(None)--> 3
0 --(a)--> 1
1 --(None)--> 2
1 --(None)--> 3
